In [1]:
from sympy import primerange, sqrt
from decimal import Decimal, getcontext
from math import isclose

# Set precision high enough for sqrt of large numbers
getcontext().prec = 150

# Generate the list of 100 large integers: (10^100) * sqrt(p_i)
def generate_large_numbers():
    primes = list(primerange(1, 550))[:100]  # First 100 primes
    scale = Decimal(10) ** 100
    items = [int(scale * Decimal(str(sqrt(p).evalf()))) for p in primes]  # Convert Float to Decimal via str
    return items

# FPTAS implementation
def approx_subset_sum(items, target, eps):
    items = sorted(items)
    n = len(items)
    L = [0]
    for x in items:
        new_sums = [s + x for s in L]
        U = sorted(set(L + new_sums))
        trimmed = []
        last = U[0]
        trimmed.append(last)
        for z in U[1:]:
            if z > last * (1 + eps / n):
                trimmed.append(z)
                last = z
        L = [s for s in trimmed if s <= target]
    return L[-1] if L else 0

# Generate the 100-item list
items = generate_large_numbers()

# Example usage — you define the target!
target = 7 * 113596441 * 10**94  # Example target
epsilon = 0.0001  # e.g. 1% error tolerance

# Measure runtime
import time
start_time = time.time()

# Compute the approximate subset sum
result = approx_subset_sum(items, target, epsilon)

# Measure runtime
end_time = time.time()
runtime = end_time - start_time

# Calculate the error and its number of digits
error = abs(target - result)
num_digits_in_error = len(str(error))

# Print the results
print(f"Approximate subset sum (ε={epsilon}):\n{result}")
print(f"Target sum:\n{target}")
print(f"Error (difference):\n{error}")
print(f"Number of digits in the error:\n{num_digits_in_error}")
print(f"Time taken: {runtime:.2f} seconds")

ModuleNotFoundError: No module named 'sympy'

In [ ]:
# worse
def subset_sum_alternate(M: list[int], S: int) -> list[int]:
    """
    M is a list (length 100) of integers and S is the target sum.
    Return the subset that sums the closest to S.
    """
    T = [0] * len(M) # binary list to show if the element from M is in the subset

    # Choose the first ele, then the last, then the second, then the second to last, etc.
    # This is a bit of a hack, but it works for this problem.
    for i in range(len(M) // 2):
        if sum(M[j] for j in range(len(M)) if T[j]) + M[i] <= S:
            T[i] = 1
        if sum(M[j] for j in range(len(M)) if T[j]) + M[len(M) - 1 - i] <= S:
            T[len(M) - 1 - i] = 1

    return T


def subset_sum_refine_3(M: list[int], S: int, subset_sum_function: Callable, k_l_pairs: List[tuple], N=None) -> list[int]:
    """
    M is a list (length 100) of integers and S is the target sum.
    Return the subset that sums the closest to S.
    Calls a different function to start this process, then refines the result.
    k_l_pairs: A list of (k, l) pairs to use for each iteration.
    N: Number of iterations (optional). If not provided, defaults to the length of k_l_pairs.
    """

    if N is None:
        N = len(k_l_pairs)

    print(f"Refining with {N} iterations and varying k, l pairs", flush=True)
    T0 = subset_sum_function(M, S)
    sum0 = sum_from_bin_list(T0, M)
    err = abs(S - sum0)

    for i in range(N):
        k, l = k_l_pairs[i % len(k_l_pairs)]  # Cycle through k_l_pairs if N > len(k_l_pairs)
        print(f"Iteration {i + 1} of {N}: Using k = {k}, l = {l}", flush=True)
        improve = True
        while improve:
            improve = False
            # Generate all combinations of k elements to remove
            remove_indices = [i for i in range(len(M)) if T0[i] == 1]
            add_indices = [i for i in range(len(M)) if T0[i] == 0]

            from itertools import combinations

            for remove_comb in combinations(remove_indices, k):
                for add_comb in combinations(add_indices, l):
                    # Calculate the new sum after removing and adding elements
                    new_sum = sum0
                    for r in remove_comb:
                        new_sum -= M[r]
                    for a in add_comb:
                        new_sum += M[a]

                    new_err = abs(S - new_sum)
                    if new_err < err:
                        # Update the subset and the error
                        print(f"Improving: removing {remove_comb} and adding {add_comb}", flush=True)
                        for r in remove_comb:
                            T0[r] = 0
                        for a in add_comb:
                            T0[a] = 1
                        sum0 = new_sum
                        err = new_err
                        improve = True
                        break
                if improve:
                    break

    return T0

def timer(func: Callable) -> Callable:
    """
    Decorator to time a function.
    """
    def wrapper(*args, **kwargs):
        import time
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Time taken: {end_time - start_time:.2f} seconds")
        return result
    return wrapper

@timer
def main(subset_sum_function=subset_sum_backwards, ID=113596441, extra_args=[]):
    print(f"Using {subset_sum_function.__name__} with ID {ID}")
    print()
    # generate the first 100 prime numbers
    primes = []
    i = 2
    while len(primes) < 100:
        if is_prime(i):
            primes.append(i)
        i += 1

    # print(len(primes))
    # print(primes)
    # print()

    # M = [10**100 * i**0.5 for i in primes]
    M = [math.floor(Decimal(i).sqrt() * Decimal(10**100)) for i in primes]
    # print(len(M))
    # print(M)

    S = 7 * ID * 10**94

    print("Number of digits in the target sum:", len(str(int(S))))
    print("Target sum:", S)
    print()

    exp_target = sum_from_bin_list(subset_sum_function(M, S, *extra_args), M)
    print("Number of digits in the subset sum:", len(str(int(exp_target))))
    print("Subset sum:", exp_target)
    print()

    print("Number of digits in the difference:", len(str(int(S - exp_target))))
    print("Difference:", S - exp_target) 
    print()

main(subset_sum_function=subset_sum_refine_3, ID=ID, extra_args=[subset_sum_alternate, [(1, 1), (1, 2), (2, 3), (3, 3), (3, 2), (2, 1), (1, 1)]])


In [39]:
def check_binary_list(binary_list, M=None, target=None):
    """
    Check how close the sum of a binary list is to the target.
    
    Args:
        binary_list: A list of 0s and 1s indicating which elements to include
        M: The list of integers (uses the already generated one if None)
        target: The target sum (uses the already calculated one if None)
        
    Returns:
        Dictionary containing sum, error, and other information
    """
    import time
    
    if M is None:
        try:
            # Try to use the M from the notebook
            M = globals().get('M') or generate_large_numbers()
        except:
            print("Error: Could not find or generate M list")
            return
    
    if target is None:
        try:
            # Try to use the target from the notebook
            target = globals().get('target') or (7 * 113596441 * 10**94)
        except:
            print("Error: Could not find or generate target")
            return
    
    # Validate binary list
    if len(binary_list) != len(M):
        print(f"Error: Binary list length ({len(binary_list)}) must match M length ({len(M)})")
        return
    
    if not all(b in [0, 1] for b in binary_list):
        print("Error: Input must be a binary list (containing only 0s and 1s)")
        return
    
    # Calculate the sum
    start_time = time.time()
    subset_sum = sum(M[i] for i in range(len(M)) if binary_list[i])
    end_time = time.time()
    
    # Calculate the error
    error = abs(target - subset_sum)
    
    # Prepare results
    result = {
        "subset_sum": subset_sum,
        "target": target,
        "error": error,
        "error_digits": len(str(error)),
        "time": end_time - start_time,
        "num_elements": sum(binary_list),
        "percentage_error": (error / target) * 100 if target != 0 else float('inf')
    }
    
    # Print the results
    print(f"Subset sum:\n{subset_sum}")
    print(f"Target sum:\n{target}")
    print(f"Error (difference):\n{error}")
    print(f"Number of digits in error: {result['error_digits']}")
    print(f"Percentage error: {result['percentage_error']:.8f}%")
    print(f"Number of elements in subset: {result['num_elements']} out of {len(M)}")
    print(f"Time taken: {result['time']:.6f} seconds")
    
    return result
from decimal import Decimal, getcontext

import math
def is_prime(n):
    """Check if a number is prime."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

primes = []
i = 2
while len(primes) < 100:
    if is_prime(i):
        primes.append(i)
    i += 1

    # print(len(primes))
    # print(primes)
    # print()

# M = [10**100 * i**0.5 for i in primes]
M = [math.floor(Decimal(i).sqrt() * Decimal(10**100)) for i in primes]
# print(len(M))
# print(M)

ID = 113596441
S = 7 * ID * 10**94
# Example usage
# 92 digits of error
# binary_list = [0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
#                0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
#                0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
#                0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
#                1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
#                1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
#                0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
#                1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
#                0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
#                1, 1, 1, 0, 1, 1, 1, 0, 1, 1,] 
binary_list = [1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]



check_binary_list(binary_list, M, S)

Subset sum:
7951750869999985706134892659170000000000000000000000000000000000000000000000000000000000000000000000000
Target sum:
7951750870000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Error (difference):
14293865107340830000000000000000000000000000000000000000000000000000000000000000000000000
Number of digits in error: 89
Percentage error: 0.00000000%
Number of elements in subset: 54 out of 100
Time taken: 0.000006 seconds


{'subset_sum': 7951750869999985706134892659170000000000000000000000000000000000000000000000000000000000000000000000000,
 'target': 7951750870000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000,
 'error': 14293865107340830000000000000000000000000000000000000000000000000000000000000000000000000,
 'error_digits': 89,
 'time': 5.7220458984375e-06,
 'num_elements': 54,
 'percentage_error': 1.7975745645236532e-13}

In [ ]:
#!/usr/bin/env python3
"""
Version 1: Pure Python LLL-based subset-sum solver.
We embed the scaled subset-sum into a lattice and apply LLL reduction.
After reducing, we take the first basis vector, round its entries to 0/1,
and use those bits as a subset. We then compute the achieved sum and error.
"""

import math

# 1) Generate first 100 primes
def first_primes(n):
    primes = []
    num = 2
    while len(primes) < n:
        isprime = True
        for p in primes:
            if p*p > num:
                break
            if num % p == 0:
                isprime = False
                break
        if isprime:
            primes.append(num)
        num += 1
    return primes

primes = first_primes(100)

# 2) Build scaled weights M' = floor(10^6 * sqrt(p_i)), and target S' = S/10^94
M = [math.floor((10**6) * math.sqrt(p)) for p in primes]
S_target = 7 * 113596441  # Original S = this * 10^94
n = len(M)

# 3) Construct lattice basis matrix B of size (n+1)x(n+1) with scaling N
N = 1000  # scale factor to avoid trivial short vectors (Lagarias–Odlyzko trick)
# Basis is stored as a list of lists (rows of the matrix)
B = [[0.0]*(n+1) for _ in range(n+1)]
for i in range(n):
    B[i][i] = 1.0
    B[i][n] = float(N * M[i])  # place scaled weight in last column
# Last row has zeros in first n coords and -N*S in last coord
for j in range(n):
    B[n][j] = 0.0
B[n][n] = float(-N * S_target)

# 4) Define Gram-Schmidt and LLL reduction functions
def gram_schmidt(B):
    """Compute the Gram-Schmidt orthogonalization of rows of B."""
    n = len(B)
    m = len(B[0])
    # b_star will hold the orthogonalized vectors
    b_star = [[0.0]*m for _ in range(n)]
    # mu[i][j] is projection factor of B[i] on b_star[j]
    mu = [[0.0]*n for _ in range(n)]
    norm = [0.0]*n  # squared norms of b_star
    for i in range(n):
        # Start with b_star[i] = B[i]
        for k in range(m):
            b_star[i][k] = B[i][k]
        for j in range(i):
            # Compute mu[i][j] = (B[i]·b_star[j])/(||b_star[j]||^2)
            dot = sum(B[i][k] * b_star[j][k] for k in range(m))
            mu[i][j] = dot / norm[j]
            # Subtract the projection
            for k in range(m):
                b_star[i][k] -= mu[i][j] * b_star[j][k]
        # Compute squared norm of b_star[i]
        norm[i] = sum(b_star[i][k] * b_star[i][k] for k in range(m))
    return b_star, mu, norm

def lll_reduce(B, delta=0.75):
    """Perform LLL lattice basis reduction on matrix B in-place."""
    n = len(B)
    m = len(B[0])
    # Start Gram-Schmidt
    b_star, mu, norm = gram_schmidt(B)
    k = 1
    while k < n:
        # Size-reduce B[k] by previous vectors
        for j in range(k-1, -1, -1):
            if abs(mu[k][j]) > 0.5:
                r = round(mu[k][j])
                # B[k] = B[k] - r * B[j]
                for t in range(m):
                    B[k][t] -= r * B[j][t]
                # After modification, recompute GS
                b_star, mu, norm = gram_schmidt(B)
        # Check Lovász condition
        if norm[k] < (delta - mu[k][k-1]**2) * norm[k-1]:
            # Swap B[k] and B[k-1]
            B[k], B[k-1] = B[k-1], B[k]
            # Recompute GS from scratch
            b_star, mu, norm = gram_schmidt(B)
            k = max(k-1, 1)
        else:
            k += 1

# 5) Run LLL on the basis
lll_reduce(B, delta=0.75)

# 6) Extract the first basis vector and derive a subset
v = B[0]  # first reduced basis vector
# Round the first n coordinates to 0 or 1
x = []
for i in range(n):
    # If coordinate is >=0.5, treat as 1; else 0.
    if v[i] >= 0.5:
        x.append(1)
    else:
        x.append(0)

# 7) Compute subset sum and difference
chosen = [i for i,val in enumerate(x) if val == 1]
subset_sum = sum(M[i] for i in chosen)
diff = subset_sum - S_target

# 8) Output results
print("Version 1 (Pure Python LLL):")
print(f"Chosen subset indices (0-based): {chosen}")
print(f"Subset sum = {subset_sum}, Target = {S_target}, Difference = {diff}")


Version 1 (Pure Python LLL):
Chosen subset indices (0-based): [0, 1, 4, 6, 7]
Subset sum = 14944890, Target = 795175087, Difference = -780230197


In [ ]:
#!/usr/bin/env python3
"""
Version 2: Using fpylll for lattice reduction.
We build the same lattice basis as in Version 1 but as an IntegerMatrix,
call LLL.reduction, and then interpret the first reduced basis vector(s)
to get a subset solution.
"""

from fpylll import IntegerMatrix, LLL
import math

# 1) Generate first 100 primes (same as above)
def first_primes(n):
    primes = []
    num = 2
    while len(primes) < n:
        isprime = True
        for p in primes:
            if p*p > num:
                break
            if num % p == 0:
                isprime = False
                break
        if isprime:
            primes.append(num)
        num += 1
    return primes

primes = first_primes(100)

# 2) Scaled weights and target
M = [math.floor((10**6) * math.sqrt(p)) for p in primes]
S_target = 7 * 113596441
n = len(M)

# 3) Construct lattice basis as IntegerMatrix
N = 1000
B = IntegerMatrix(n+1, n+1)
# Fill basis: identity in first n columns, scaled weights in last column
for i in range(n):
    B[i, i] = 1
    B[i, n] = N * M[i]
# Last row: all zeros except last column = -N*S
for j in range(n):
    B[n, j] = 0
B[n, n] = -N * S_target

# 4) Apply LLL reduction (default δ=0.75, η=0.501)
LLL.reduction(B)

# 5) Try the first few basis vectors to find a subset
best_diff = None
best_subset = None
best_sum = None
for row in range(min(3, n+1)):
    vec = [B[row, col] for col in range(n+1)]
    # Interpret as solution: use 1 if coordinate > 0, else 0
    x = [1 if vec[i] > 0 else 0 for i in range(n)]
    subset = [i for i, xi in enumerate(x) if xi == 1]
    subset_sum = sum(M[i] for i in subset)
    diff = subset_sum - S_target
    # Keep the subset with smallest absolute difference
    if best_diff is None or abs(diff) < abs(best_diff):
        best_diff = diff
        best_subset = subset
        best_sum = subset_sum

# 6) Output results
print("\nVersion 2 (fpylll LLL):")
print(f"Chosen subset indices (0-based): {best_subset}")
print(f"Subset sum = {best_sum}, Target = {S_target}, Difference = {best_diff}")



Version 2 (fpylll LLL):
Chosen subset indices (0-based): [11, 12, 13, 15]
Subset sum = 26323433, Target = 795175087, Difference = -768851654


In [ ]:
# Load all of Sage’s symbolic and numeric environment
from sage.all import Matrix, ZZ, vector, primes, floor, sqrt

# 1) Parameters
n = 100
# First 100 primes
plist = list(primes(2, 2*n))[:n]                             # :contentReference[oaicite:0]{index=0}
# Scaled weights and target (scaled down by 10^94)
M = [floor(10^6 * sqrt(p)) for p in plist]                   # :contentReference[oaicite:1]{index=1}
S = 7 * 113596441                                            # :contentReference[oaicite:2]{index=2}

# 2) Embedding scale (power of two ≥ max(M) for good conditioning)
N = 1 << max(M).bit_length()                                 # :contentReference[oaicite:3]{index=3}

# 3) Build lattice basis: identity + weight column, and the target row
B = Matrix(ZZ, n+1, n+1)
for i in range(n):
    B[i,i] = 1
    B[i,n] = N * M[i]                                        # :contentReference[oaicite:4]{index=4}
B[n] = [0]*n + [-N * S]                                      # :contentReference[oaicite:5]{index=5}

# 4) Lattice reduction: choose either LLL or BKZ
B_lll  = B.LLL(delta=0.99)                                   # :contentReference[oaicite:6]{index=6}
B_bkz  = B.BKZ(block_size=30, proof=False)                   # :contentReference[oaicite:7]{index=7}

# 5) CVP: find the closest lattice vector to the embedding of -N⋅S
tvec = vector(ZZ, [B_bkz[i,n] for i in range(n+1)])
v    = B_bkz.closest_vector(tvec)                           # :contentReference[oaicite:8]{index=8}

# 6) Extract 0/1 subset from the first n coordinates
x = [1 if c > 0 else 0 for c in v[:n]]
chosen = [i for i, xi in enumerate(x) if xi]

# 7) Compute achieved sum and difference
subset_sum = sum(M[i] for i in chosen)
diff       = subset_sum - S

# 8) Output
print("Chosen indices:", chosen)
print("Subset sum =", subset_sum)
print("Target      =", S)
print("Difference  =", diff)


ModuleNotFoundError: No module named 'sage'